##### Copyright 2023 Phillip Truppelli

In [2]:
# The purpose of this script is to showcase how natural language converted into quantum states can be implemented in a query answering problem using Grover's algorithm.
# This will provide a baseline framework for query answering on a quantum computer.
# You may obtain the source code and documentation here:
# 
# https://github.com/truppelli
# 
# This code is covered under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector, array_to_latex
from qiskit.extensions import Initialize
from qiskit.quantum_info import random_statevector, Statevector

qr0 = QuantumRegister(1, name="q[0]")
qr1 = QuantumRegister(1, name="q[1]")
qr2 = QuantumRegister(1, name="q[2]")
qr3 = QuantumRegister(1, name="q[3]")
qr4 = QuantumRegister(1, name="q[4]")
qr5 = QuantumRegister(1, name="q[5]")
qr6 = QuantumRegister(1, name="q[6]")
qr7 = QuantumRegister(1, name="q[7]")
qr8 = QuantumRegister(1, name="q[8]")
qr9 = QuantumRegister(1, name="q[9]")
qr10 = QuantumRegister(1, name="q[10]")
cr = ClassicalRegister(10, name="cr")

Rx(0) q[0];
H q[1];
H q[2];
H q[3];
H q[4];
H q[5];
H q[6];
H q[7];
Rx(0.006) q[8];
H q[9];
H q[10];
Rz(0.002) q[0];
CRz(0.024) q[1], q[2];
CRz(0.012) q[5], q[6];
Rz(0.008) q[8];
CRz(0.02) q[9], q[10];
Rx(0.004) q[0];
H q[1];
CRz(0.026) q[2], q[3];
H q[5];
CRz(0.014) q[6], q[7];
Rx(0.01) q[8];
H q[9];
H q[10];
H q[2];
CRz(0.028) q[3], q[4];
H q[6];
H q[7];
CRz(0.022) q[9], q[10];
CRz(0.03) q[1], q[2];
H q[3];
H q[4];
CRz(0.016) q[5], q[6];
H q[9];
H q[10];
H q[1];
CRz(0.032) q[2], q[3];
H q[5];
CRz(0.018) q[6], q[7];
CX q[0], q[1];
H q[2];
CRz(0.034) q[3], q[4];
H q[6];
H q[7];
Measure q[1] --> c[7];
H q[0];
CX q[2], q[9];
H q[3];
H q[4];
CX q[7], q[8];
Measure q[8] --> c[1];
Measure q[0] --> c[6];
Measure q[9] --> c[9];
H q[2];
CX q[3], q[6];
CX q[4], q[5];
H q[7];
Measure q[7] --> c[0];
Measure q[5] --> c[3];
Measure q[6] --> c[5];
Measure q[2] --> c[8];
H q[3];
H q[4];
Measure q[4] --> c[2];
Measure q[3] --> c[4];


In [19]:
from pytket.extensions.qiskit import tk_to_qiskit

print("{}:\n{}\n".format(tket_circuit, '\n'.join(map(str, tket_circuit))))
print("post selection:\n{}\n".format(tket_circuit.post_selection))
print("scalar:\n{}\n".format(tket_circuit.scalar))
print("qiskit circuit:")
tk_to_qiskit(tket_circuit).draw()

<tket::Circuit, qubits=11, gates=65>:
Rx(0) q[0];
H q[1];
H q[2];
H q[3];
H q[4];
H q[5];
H q[6];
H q[7];
Rx(0.006) q[8];
H q[9];
H q[10];
Rz(0.002) q[0];
CRz(0.024) q[1], q[2];
CRz(0.012) q[5], q[6];
Rz(0.008) q[8];
CRz(0.02) q[9], q[10];
Rx(0.004) q[0];
H q[1];
CRz(0.026) q[2], q[3];
H q[5];
CRz(0.014) q[6], q[7];
Rx(0.01) q[8];
H q[9];
H q[10];
H q[2];
CRz(0.028) q[3], q[4];
H q[6];
H q[7];
CRz(0.022) q[9], q[10];
CRz(0.03) q[1], q[2];
H q[3];
H q[4];
CRz(0.016) q[5], q[6];
H q[9];
H q[10];
H q[1];
CRz(0.032) q[2], q[3];
H q[5];
CRz(0.018) q[6], q[7];
CX q[0], q[1];
H q[2];
CRz(0.034) q[3], q[4];
H q[6];
H q[7];
Measure q[1] --> c[7];
H q[0];
CX q[2], q[9];
H q[3];
H q[4];
CX q[7], q[8];
Measure q[8] --> c[1];
Measure q[0] --> c[6];
Measure q[9] --> c[9];
H q[2];
CX q[3], q[6];
CX q[4], q[5];
H q[7];
Measure q[7] --> c[0];
Measure q[5] --> c[3];
Measure q[6] --> c[5];
Measure q[2] --> c[8];
H q[3];
H q[4];
Measure q[4] --> c[2];
Measure q[3] --> c[4];

post selection:
{0: 0, 1: 0, 2

┌───────┐    ┌───────────┐   ┌───────────┐                  »
 q_0: ───┤ Rx(0) ├────┤ Rz(π/500) ├───┤ Rx(π/250) ├──────────────────»
         └─┬───┬─┘    └───────────┘   └───┬───┬───┘                  »
 q_1: ─────┤ H ├────────────■─────────────┤ H ├──────────────────────»
           ├───┤     ┌──────┴───────┐     └───┘           ┌───┐      »
 q_2: ─────┤ H ├─────┤ Rz(0.075398) ├───────■─────────────┤ H ├──────»
           ├───┤     └──────────────┘┌──────┴───────┐     └───┘      »
 q_3: ─────┤ H ├─────────────────────┤ Rz(0.081681) ├───────■────────»
           ├───┤                     └──────────────┘┌──────┴───────┐»
 q_4: ─────┤ H ├─────────────────────────────────────┤ Rz(0.087965) ├»
           ├───┤                          ┌───┐      └──────────────┘»
 q_5: ─────┤ H ├────────────■─────────────┤ H ├──────────────────────»
           ├───┤     ┌──────┴───────┐     └───┘           ┌───┐      »
 q_6: ─────┤ H ├─────┤ Rz(0.037699) ├───────■─────────────┤ H ├──────»
           ├───┤     └──────────────┘┌──────┴───────┐     ├───┤      »
 q_7: ─────┤ H ├─────────────────────┤ Rz(0.043982) ├─────┤ H ├──────»
      ┌────┴───┴────┐ ┌───────────┐  └┬───────────┬─┘     └───┘      »
 q_8: ┤ Rx(0.01885) ├─┤ Rz(π/125) ├───┤ Rx(π/100) ├──────────────────»
      └────┬───┬────┘ └───────────┘   └───┬───┬───┘                  »
 q_9: ─────┤ H ├────────────■─────────────┤ H ├─────────────■────────»
           ├───┤       ┌────┴─────┐       ├───┤      ┌──────┴───────┐»
q_10: ─────┤ H ├───────┤ Rz(π/50) ├───────┤ H ├──────┤ Rz(0.069115) ├»
           └───┘       └──────────┘       └───┘      └──────────────┘»
c: 10/═══════════════════════════════════════════════════════════════»
                                                                     »
«                                                     ┌───┐               ┌─┐»
« q_0: ───────────────────────────────────────■───────┤ H ├───────────────┤M├»
«                           ┌───┐           ┌─┴─┐     └┬─┬┘               └╥┘»
« q_1: ───────■─────────────┤ H ├───────────┤ X ├──────┤M├─────────────────╫─»
«      ┌──────┴───────┐     └───┘           ├───┤      └╥┘      ┌───┐      ║ »
« q_2: ┤ Rz(0.094248) ├───────■─────────────┤ H ├───────╫────■──┤ H ├──────╫─»
«      └────┬───┬─────┘┌──────┴──────┐      └───┘       ║    │  ├───┤      ║ »
« q_3: ─────┤ H ├──────┤ Rz(0.10053) ├────────■─────────╫────┼──┤ H ├──■───╫─»
«           ├───┤      └─────────────┘ ┌──────┴──────┐  ║    │  ├───┤  │   ║ »
« q_4: ─────┤ H ├──────────────────────┤ Rz(0.10681) ├──╫────┼──┤ H ├──┼───╫─»
«           └───┘           ┌───┐      └─────────────┘  ║    │  └───┘  │   ║ »
« q_5: ───────■─────────────┤ H ├───────────────────────╫────┼─────────┼───╫─»
«      ┌──────┴───────┐     └───┘           ┌───┐       ║    │       ┌─┴─┐ ║ »
« q_6: ┤ Rz(0.050265) ├───────■─────────────┤ H ├───────╫────┼───────┤ X ├─╫─»
«      └──────────────┘┌──────┴───────┐     ├───┤       ║    │       ├───┤ ║ »
« q_7: ────────────────┤ Rz(0.056549) ├─────┤ H ├───────╫────┼────■──┤ H ├─╫─»
«                      └──────────────┘     └───┘       ║    │  ┌─┴─┐└┬─┬┘ ║ »
« q_8: ─────────────────────────────────────────────────╫────┼──┤ X ├─┤M├──╫─»
«           ┌───┐                                       ║  ┌─┴─┐└┬─┬┘ └╥┘  ║ »
« q_9: ─────┤ H ├───────────────────────────────────────╫──┤ X ├─┤M├───╫───╫─»
«           ├───┤                                       ║  └───┘ └╥┘   ║   ║ »
«q_10: ─────┤ H ├───────────────────────────────────────╫─────────╫────╫───╫─»
«           └───┘                                       ║         ║    ║   ║ »
«c: 10/═════════════════════════════════════════════════╩═════════╩════╩═══╩═»
«                                                       7         9    1   6 »
«                            
« q_0: ──────────────────────
«                            
« q_1: ──────────────────────
«                   ┌─┐      
« q_2: ─────────────┤M├──────
«      ┌───┐        └╥┘   ┌─┐
« q_3: ┤ H ├─────────╫────┤M├
«      └───┘┌───┐  